In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sklearn.metrics
import itertools

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [4]:

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4

In [5]:

def accuracy(predictions,labels):
    '''
    Accuracy of a given set of predictions of size (N x n_classes) and
    labels of size (N x n_classes)
    '''
    return np.sum(np.argmax(predictions,axis=1)==np.argmax(labels,axis=1))*100.0/labels.shape[0]


def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [6]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

In [7]:
def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)

  return figure

In [8]:
def log_confusion_matrix(epoch, logs, model, test_images,test_labels, class_names,file_writer_cm):

 # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [9]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. 
    
    `images` should be a list of all of the images in the data directory, 
    where each image is formatted as a numpy ndarray with 
    dimensions IMG_WIDTH x IMG_HEIGHT x 3. 
    
    `labels` should be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels = []
    labels_str= []
    filenames = []
    for dir in os.listdir(data_dir):
        # get path and filename
        if dir[0] != ".":
            path = os.path.join(data_dir, dir)
            category = path.split(os.sep)[-1:]
            filename = os.listdir(path)
            
            # get img and scale it to 30,30
            for fname in filename:
                filepath = os.path.join(path,fname)
                
                img = cv2.imread(filepath)
                
                if img is None:
                    sys.exit("Could not read the image.")
                
                res = cv2.resize(img,(IMG_HEIGHT,IMG_WIDTH),interpolation = cv2.INTER_AREA)
                
                # add to image list and category list
                images.append(res)
                labels.append(int(category[0]))
                labels_str.append(category[0])
                filenames.append(fname)

    # print("NumIng", len(images),"NumLab", len(labels))
    # print("Img shape", images[0].shape)
    # print("Labels types: ", type(labels[0]))
   
    return (images,labels,labels_str)

# Get image arrays and labels for all image files
images, labels, class_names = load_data("/Volumes/Seagate/Databases/gtsrb")


In [10]:
counter = {}

for letter in labels:
     if letter not in counter:
         counter[letter] = 0
     counter[letter] += 1
counter


{0: 150,
 1: 1500,
 10: 1350,
 11: 900,
 12: 1410,
 13: 1440,
 14: 540,
 15: 420,
 16: 300,
 17: 750,
 18: 810,
 19: 150,
 2: 1500,
 20: 240,
 21: 240,
 22: 270,
 23: 360,
 24: 180,
 25: 1020,
 26: 420,
 27: 180,
 28: 360,
 29: 180,
 3: 960,
 30: 300,
 31: 540,
 32: 180,
 33: 480,
 34: 300,
 35: 810,
 36: 270,
 37: 150,
 38: 1380,
 39: 210,
 4: 1320,
 40: 240,
 41: 180,
 42: 180,
 5: 1260,
 6: 300,
 7: 960,
 8: 960,
 9: 990}

In [11]:
# Check command-line arguments
# if len(sys.argv) not in [2, 3]:
#     sys.exit("Usage: python traffic.py data_directory [model.h5]")
# converting vector of integer to binary class matrix
labels = tf.keras.utils.to_categorical(labels)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE
)

In [12]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [13]:
x_train[0]
cv2.imshow("DISPLAY IMG", x_train[0])
cv2.waitKey(60)

-1

In [14]:
# Get a compiled neural network. THE MODEL
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        16,(3,3),activation="relu",input_shape=(IMG_WIDTH,IMG_HEIGHT,3)
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(64,(3,3),activation = "relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()
    return model

model = get_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)        9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               590080    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0

In [15]:
#create logs FIT directory
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# a callback for datetime directory
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [17]:
# Fit model on training data
history = model.fit(x_train, y_train, epochs=EPOCHS,validation_data=(x_test, y_test), callbacks=[tensorboard_callback])
# # Evaluate neural network performance
print("start evaluation on test data...")
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/10
500/500 [==============================] - 7s 13ms/step - loss: 2.0210 - accuracy: 0.6269 - val_loss: 0.4352 - val_accuracy: 0.8771
Epoch 2/10
500/500 [==============================] - 6s 12ms/step - loss: 0.3459 - accuracy: 0.9054 - val_loss: 0.2150 - val_accuracy: 0.9449
Epoch 3/10
500/500 [==============================] - 6s 11ms/step - loss: 0.1849 - accuracy: 0.9480 - val_loss: 0.1934 - val_accuracy: 0.9474
Epoch 4/10
500/500 [==============================] - 6s 11ms/step - loss: 0.1189 - accuracy: 0.9660 - val_loss: 0.1288 - val_accuracy: 0.9670
Epoch 5/10
500/500 [==============================] - 6s 12ms/step - loss: 0.1071 - accuracy: 0.9697 - val_loss: 0.1691 - val_accuracy: 0.9575
Epoch 6/10
500/500 [==============================] - 6s 12ms/step - loss: 0.1119 - accuracy: 0.9694 - val_loss: 0.1770 - val_accuracy: 0.9557
Epoch 7/10
500/500 [==============================] - 6s 12ms/step - loss: 0.0863 - accuracy: 0.9771 - val_loss: 0.1093 - val_accuracy: 0.9742

[0.14916586875915527, 0.9701576828956604]

In [20]:
# predictions
predictions = model.predict(x_test, verbose=1)
# predictions for 4 first images must be label 0
for i in range(4):
    print(f"Prediction ",i, "guess =", np.argmax(predictions[i]),"probabality = ", max(predictions[i]))
    print("Label ", i,"actual_category =", np.argmax(y_test[i]))

333/333 [==============================] - 1s 3ms/step
Prediction  0 guess = 1 probabality =  1.0
Label  0 actual_category = 1
Prediction  1 guess = 2 probabality =  0.9999968
Label  1 actual_category = 2
Prediction  2 guess = 34 probabality =  1.0
Label  2 actual_category = 34
Prediction  3 guess = 10 probabality =  1.0
Label  3 actual_category = 10


In [21]:
# Save model to file
filename = "model_5thJul"
model.save(filename)
print(f"Model saved to {filename}.")


INFO:tensorflow:Assets written to: model_5thJul/assets
Model saved to model_5thJul.


In [22]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 21461), started 1 day, 1:52:56 ago. (Use '!kill 21461' to kill it.)